In [1]:
import numpy as np
from tqdm import tqdm
from scipy import stats

In [23]:
BH_percentiles = [1e-2, 1e-3, 1e-4]
fixed_cut = [0.51, 0.53, 0.55]

def calc_and_apply_threshold(samples_preds, data_preds, efficiency):
    """
    Returns number of samples and data events before and after cut

    Apply quantile cut based on efficiency to samples classifier scores and then the
    same threshold to data classifier scores 
    """
    #print(1-efficiency)
    #samples_preds = np.where(samples_preds==np.nan, 0, samples_preds)
    #data_preds = np.where(data_preds==np.nan, 0, data_preds)
    #print(samples_preds.shape)
    eps = np.quantile(samples_preds, 1-efficiency, method="nearest")
    #print(eps)
    if efficiency == 1:
        eps=0.
    N_samples_after = np.size(np.where(samples_preds>eps))
    N_samples = len(samples_preds)
    N_after = np.size(np.where(data_preds>eps))
    N = len(data_preds)
    #print(N_samples_after, N_samples, N_after, N)
    return N_samples_after, N_samples, N_after, N

def apply_fixed_cut(samples_preds, data_preds, eps):
    """
    Returns number of samples and data events before and after cut

    Apply quantile cut based on efficiency to samples classifier scores and then the
    same threshold to data classifier scores 
    """

    samples_preds = np.where(samples_preds==np.nan, 0, samples_preds)
    data_preds = np.where(data_preds==np.nan, 0, data_preds)
    N_samples_after = np.size(np.where(samples_preds>=eps))
    N_samples = len(samples_preds)
    N_after = np.size(np.where(data_preds>=eps))
    N = len(data_preds)
    return N_samples_after, N_samples, N_after, N

def make_arrays(folder, start_runs=0, runs=2100):
    arr_shape = (runs,len(BH_percentiles))
    N_samples_after = np.zeros(arr_shape)
    N_samples_after_fixed = np.zeros(arr_shape)
    N_samples = np.zeros(arr_shape)
    N_after = np.zeros(arr_shape)
    N_after_fixed = np.zeros(arr_shape)
    N = np.zeros(arr_shape)


    for r in tqdm(range(start_runs, runs)):
        f = folder+"run"+str(r)+"/"
        samples_preds = np.load(f+"BT_preds.npy")[:,1]
        data_preds = np.load(f+"data_preds.npy")[:,1]
        for j, perc in enumerate(BH_percentiles):
            N_samples_after[r,j], N_samples[r,j], N_after[r,j], N[r,j] = calc_and_apply_threshold(samples_preds, data_preds, perc)
        for j, cut in enumerate(fixed_cut):
            N_samples_after_fixed[r,j], _, N_after_fixed[r,j], _ = apply_fixed_cut(samples_preds, data_preds, cut)
    np.save(folder+"N_samples_after.npy", N_samples_after)
    np.save(folder+"N_samples_after_fixed.npy", N_samples_after_fixed)
    np.save(folder+"N_samples.npy", N_samples)
    np.save(folder+"N_after.npy", N_after)
    np.save(folder+"N_after_fixed.npy", N_after_fixed)
    np.save(folder+"N.npy", N)


In [3]:
def make_arrays_shifted(folder, start_runs=0, runs=2100):
    arr_shape = (runs,len(BH_percentiles))
    N_samples_after = np.zeros(arr_shape)
    N_samples_after_fixed = np.zeros(arr_shape)
    N_samples = np.zeros(arr_shape)
    N_after = np.zeros(arr_shape)
    N_after_fixed = np.zeros(arr_shape)
    N = np.zeros(arr_shape)


    for r in tqdm(range(start_runs, runs)):
        f = folder+"run"+str(r)+"/"
        samples_preds = np.load(f+"BT_preds.npy")
        d = np.load(f+"data_preds.npy")
        data_preds = d[:10000]
        samples_preds = np.append(samples_preds, d[10000:])
        for j, perc in enumerate(BH_percentiles):
            N_samples_after[r,j], N_samples[r,j], N_after[r,j], N[r,j] = calc_and_apply_threshold(samples_preds, data_preds, perc)
        for j, cut in enumerate(fixed_cut):
            N_samples_after_fixed[r,j], _, N_after_fixed[r,j], _ = apply_fixed_cut(samples_preds, data_preds, cut)
    np.save(folder+"shifted_N_samples_after.npy", N_samples_after)
    np.save(folder+"shifted_N_samples_after_fixed.npy", N_samples_after_fixed)
    np.save(folder+"shifted_N_samples.npy", N_samples)
    np.save(folder+"shifted_N_after.npy", N_after)
    np.save(folder+"shifted_N_after_fixed.npy", N_after_fixed)
    np.save(folder+"shifted_N.npy", N)


In [5]:
make_arrays("/hpcwork/zu992399/look_elsewhere/NN_calibration/", runs=10000)
#make_arrays_shifted("/hpcwork/zu992399/look_elsewhere/NN_calibration/")

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [08:35<00:00, 19.40it/s]


In [3]:
make_arrays("/hpcwork/zu992399/look_elsewhere/BDT_calibration_dims10_bins5/", runs=10000)
#make_arrays_shifted("/hpcwork/zu992399/look_elsewhere/NN_calibration/")

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [11:44<00:00, 14.20it/s]


In [25]:
make_arrays("/hpcwork/zu992399/look_elsewhere/BDT_calibration_bins255/", runs=10000)

100%|██████████| 10000/10000 [14:19<00:00, 11.64it/s]


In [24]:
make_arrays("/hpcwork/zu992399/look_elsewhere/BDT_calibration_GBDT/", runs=10000)

100%|██████████| 10000/10000 [13:28<00:00, 12.37it/s] 


In [21]:
BH_percentiles = [1e-2, 1e-3, 1e-4]
fixed_cut = [0.51, 0.53, 0.55]

def calc_and_apply_threshold_return(samples_preds, data_preds, efficiency):
    """
    Returns number of samples and data events before and after cut

    Apply quantile cut based on efficiency to samples classifier scores and then the
    same threshold to data classifier scores 
    """
    #print(1-efficiency)
    #samples_preds = np.where(samples_preds==np.nan, 0, samples_preds)
    #data_preds = np.where(data_preds==np.nan, 0, data_preds)
    #print(samples_preds.shape)
    eps = np.quantile(samples_preds, 1-efficiency, method="nearest")
    #print(eps)
    if efficiency == 1:
        eps=0.
    N_samples_after = np.size(np.where(samples_preds>eps))
    N_samples = len(samples_preds)
    N_after = np.size(np.where(data_preds>eps))
    N = len(data_preds)
    #print(N_samples_after, N_samples, N_after, N)
    return N_samples_after, N_samples, N_after, N, eps

def apply_fixed_cut_not_samples(data_preds, eps):
    """
    Returns number of samples and data events before and after cut

    Apply quantile cut based on efficiency to samples classifier scores and then the
    same threshold to data classifier scores 
    """

    #samples_preds = np.where(samples_preds==np.nan, 0, samples_preds)
    data_preds = np.where(data_preds==np.nan, 0, data_preds)
    #N_samples_after = np.size(np.where(samples_preds>=eps))
    #N_samples = len(samples_preds)
    N_after = np.size(np.where(data_preds>=eps))
    N = len(data_preds)
    return N_after, N

def make_arrays_fixed_threshold(folder, start_runs=0, runs=2100):
    arr_shape = (runs,len(BH_percentiles))
    N_samples_after = np.zeros(arr_shape)
    N_samples = np.zeros(arr_shape)
    N_after = np.zeros(arr_shape)
    N = np.zeros(arr_shape)

    eps = np.zeros(len(BH_percentiles))
    for r in tqdm(range(start_runs, runs)):
        f = folder+"run"+str(r)+"/"
        samples_preds = np.load(f+"BT_preds.npy")[:,1]
        data_preds = np.load(f+"data_preds.npy")[:,1]
        if r==0:
            for j, perc in enumerate(BH_percentiles):
                N_samples_after[r,j], N_samples[r,j], N_after[r,j], N[r,j], eps[j] = calc_and_apply_threshold_return(samples_preds, data_preds, perc)
        else:
            for j, cut in enumerate(eps):
                N_after[r,j],N[r,j] = apply_fixed_cut_not_samples(data_preds, cut)
                N_samples_after[r,j] = N_samples_after[0,j]
                N_samples[r,j] = N_samples[0,j]

    np.save(folder+"N_samples_after.npy", N_samples_after)
    np.save(folder+"N_samples.npy", N_samples)
    np.save(folder+"N_after.npy", N_after)
    np.save(folder+"N.npy", N)


In [22]:
make_arrays_fixed_threshold("/hpcwork/zu992399/look_elsewhere/BDT_calibration_only_change_data/", runs=1000)

100%|██████████| 1000/1000 [04:00<00:00,  4.16it/s]


In [4]:
def p_value_poissonpoisson(N_data, N_BT, k=0.5):
    return 1-stats.nbinom.cdf(N_data-1, N_BT, k)

def make_arrays_signal(folder, positions, widths, numbers, start_runs=0, runs=10):
    arr_shape = (len(positions),len(widths), len(numbers),runs,len(BH_percentiles))
    N_samples_after = np.zeros(arr_shape)
    N_samples = np.zeros(arr_shape)
    N_after = np.zeros(arr_shape)
    N = np.zeros(arr_shape)
    pvalues = np.zeros(arr_shape)

    for i,p in enumerate(positions):
        print(p)
        if p.is_integer():
            p = int(p)
        for j,w in enumerate(widths):
            if w.is_integer():
                w = int(w)
            for k,s in enumerate(numbers):
                for m in range(runs):
                    f = folder+"pos"+str(p)+"_width"+str(w)+"_Nsig"+str(s)+"/run"+str(m)+"/"
                    samples_preds = np.load(f+"BT_preds.npy")[:,1]
                    data_preds = np.load(f+"data_preds.npy")[:,1]
                    for l, perc in enumerate(BH_percentiles):
                        N_samples_after[i,j,k,m,l], N_samples[i,j,k,m,l], N_after[i,j,k,m,l], N[i,j,k,m,l] = calc_and_apply_threshold(samples_preds, data_preds, perc)
                        pvalues[i,j,k,m,l] = p_value_poissonpoisson(N_after[i,j,k,m,l], N_samples_after[i,j,k,m,l])
    np.save(folder+"N_samples_after.npy", N_samples_after)
    np.save(folder+"N_samples.npy", N_samples)
    np.save(folder+"N_after.npy", N_after)
    np.save(folder+"N.npy", N)
    np.save(folder+"pvalues.npy", pvalues)

In [5]:
signal_positions = np.arange(1, 3.1, 0.5)
signal_widths = np.array([0.01, 0.02, 0.03, 0.04, 0.05, 0.075, 0.1, 0.2, 0.3, 0.4, 0.5, 1.])
N_sig = np.array([50,100,200, 500, 1000])#, 200, 500, 1000, 2000, 5000])
parent_folder = "/hpcwork/zu992399/look_elsewhere/NN_signal/"
make_arrays_signal(parent_folder, signal_positions, signal_widths, N_sig)

1.0
1.5
2.0
2.5
3.0
